In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/soc-sign-bitcoinotc.csv')

In [3]:
data

,user,other,evaluation,weight
0,6,2,4,1.289242e+09
1,6,5,2,1.289242e+09
2,1,15,1,1.289243e+09
3,4,3,7,1.289245e+09
4,13,16,8,1.289254e+09
5,13,10,8,1.289254e+09
6,7,5,1,1.289363e+09
7,2,21,5,1.289371e+09
8,2,20,5,1.289371e+09
9,21,2,5,1.289381e+09
